In [ ]:
import re
import cv2
import os
import numpy
import time
import pandas
from IPython.display import HTML
from PIL import Image, ImageDraw, ImageFont

def cv2_to_image(img: numpy.ndarray) -> Image:
    return Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

def image_to_cv2(img: Image) -> numpy.ndarray:
    return cv2.cvtColor(numpy.array(img), cv2.COLOR_RGB2BGR)

In [ ]:
def save_icon(set_name, char_num, name):
    width = 212
    height = 212
    f1 = ImageFont.truetype(f"/System/Library/Fonts/Supplemental/{set_name}.ttf", 150, encoding="symb")
    img = Image.new('RGBA', (width, height), color=(255, 255, 255, 0))

    draw = ImageDraw.Draw(img)
    draw.text((10, 10), chr(char_num), font=f1, fill=(25, 25, 25, 255))    
    
    gray = cv2.cvtColor(numpy.array(img), cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    try:
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        x,y,w,h = cv2.boundingRect(numpy.vstack(contours))
    except:
        return
    l_long = max(w, h)
    l_short = min(w, h)
    l_diff = l_long - l_short
    if w > h:
        y = int(y - l_diff / 2)
    else:
        x = int(x - l_diff / 2)
    img = img.crop((x, y, x+l_long, y+l_long))
    file_name = re.sub(r"[^a-z0-9]+", "_", name.lower())
    shape_path = f"Tableau-Shapes/{set_name}"
    if not os.path.exists(shape_path):
        os.makedirs(shape_path)
    img.save(f'{shape_path}/{char_num}_{file_name}.png')


In [ ]:
for char_set in ['Webdings', 'Wingdings', 'Wingdings 2', 'Wingdings 3']:
    url = "http://www.alanwood.net/demos/%s.html" % re.sub(r" ", "-", char_set.lower())
    print(url)
    tables = pandas.read_html(url)
    for i, d in tables[0].iterrows():
        print("%s - %s - %s" % (char_set, int(d[1]), chr(d[1])))
        save_icon(char_set, int(d[1]), d[3])
